## https://cookbook.openai.com/examples/get_embeddings_from_dataset

# Get embeddings from dataset

This notebook gives an example on how to get embeddings from a large dataset.


## 1. Load the dataset

The dataset used in this example is [fine-food reviews](https://www.kaggle.com/snap/amazon-fine-food-reviews) from Amazon. The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. We will use a subset of this dataset, consisting of 1,000 most recent reviews for illustration purposes. The reviews are in English and tend to be positive or negative. Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text).

We will combine the review summary and review text into a single combined text. The model will encode this combined text and it will output a single vector embedding.

To run this notebook, you will need to install: pandas, openai, transformers, plotly, matplotlib, scikit-learn, torch (transformer dep), torchvision, and scipy.

In [1]:
import pandas as pd
import tiktoken
import openai
import os
import dotenv
import openai

from openai import AzureOpenAI



In [2]:

# Set up OpenAI client based on environment variables
dotenv.load_dotenv()
AZURE_API_KEY = os.getenv("AZURE_API_KEY")
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")

# api_version = "2023-05-15"
api_version = "2024-06-01"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_API_KEY,
)

In [3]:
embedding_model = "text-embedding-3-small"
embedding_encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

In [4]:
from typing import List
def get_embedding(text: str, model="text-embedding-3-small", **kwargs) -> List[float]:
    # replace newlines, which can negatively affect performance.
    text = text.replace("\n", " ")

    response = client.embeddings.create(input=[text], model=model, **kwargs)

    return response.data[0].embedding

In [5]:
%%time

get_embedding("This is an example sentence", model="text-embedding-3-small")

CPU times: user 142 ms, sys: 28.9 ms, total: 170 ms
Wall time: 387 ms


NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

In [ ]:
# load & inspect dataset
input_datapath = "./data/fine_food_reviews_1k.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)

In [ ]:
%%time

# subsample to 1k most recent reviews and remove samples that are too long
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)

df.head(2)

## 2. Get embeddings and save them for future reuse

In [7]:
%%time

# Ensure you have your API key set in your environment per the README: https://github.com/openai/openai-python#usage

# This may take a few minutes
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=embedding_model))
df.to_csv("./data/fine_food_reviews_with_embeddings_1k.csv")

CPU times: user 9 μs, sys: 1 μs, total: 10 μs
Wall time: 14.1 μs


NameError: name 'df' is not defined

In [8]:
a = get_embedding("hi", model=embedding_model)

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

In [ ]:
print(a)

In [ ]:
df.head(2)

In [ ]:
#save df to pkl file
df.to_pickle('./data/fine_food_reviews_with_embeddings_1.pkl')
print("done")